In [110]:
import pandas as pd
from fuzzywuzzy import fuzz
import time 

In [111]:
# All stores 

stores = ['zehrs', 'no_frills', 'valu_mart']

all_data = pd.DataFrame()


# search all stores 
for store in stores:
    regular_priced = pd.read_csv(f'clean_data/{store}/regular_prices.csv')
    flyer = pd.read_csv(f'clean_data/{store}/flyer_deals.csv')
    

In [112]:
# https://newscatcherapi.com/blog/ultimate-guide-to-text-similarity-with-python

def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

In [118]:
start_time = time.time()

store = 'zehrs'

regular_priced = pd.read_csv(f'clean_data/{store}/regular_prices.csv')
flyer = pd.read_csv(f'clean_data/{store}/flyer_deals.csv')


# important to put descriptive items on you list - 2% milk vs milk, cheddar cheese vs cheddar cheese slices 
# the more descriptive the more likely you are to get what you're looking for 
# even adding Old/Medium to Cheddar cheese makes it more descriptive 

# in the data, it is likely that the items are also ordered by relevance (ie: cheddar cheese block vs slices)
# could do a secondary ordering by index

grocery_list = ['2% milk', 'Cheddar Cheese', 'white sliced bread', 'ground beef']


final_selection = pd.DataFrame(columns = ['store', 'product_name', 'price'])

for item in grocery_list: 
    item_selection = pd.DataFrame(columns = ['store', 'product_name', 'price'])

    for index, row in regular_priced.iterrows():
        product_name = row['product']
        full_product = row['product_name']
        per_unit_price = row['per_unit_price2']
        price = row['price2']

        # find items  
        similarity = jaccard_similarity(item.lower().split(' '), product_name.lower().split(' '))

        if similarity >= 0.5: # can tweak threshold but this is a good one for now  
            data = { 'store':store, 'product_name':product_name, 'full_product':full_product, 'price':price, 'per_unit_price':per_unit_price, 'simmilarity':similarity }
            item_selection = item_selection.append(data, ignore_index=True)

    # find lowest price from top similarities
    cheapest_item = item_selection.sort_values(by=['per_unit_price'])

    final_selection = final_selection.append(dict(cheapest_item.iloc[0]), ignore_index=True)
    
    
print(time.time() - start_time, 'seconds')

final_selection

0.8788061141967773 seconds


,store,product_name,price,full_product,per_unit_price,simmilarity
0,zehrs,2% Milk,4.99,Neilson2% Milk2 l,0.0025,1.000000
1,zehrs,Cheddar Cheese Thin Slices,4.79,Black DiamondCheddar Cheese Thin Slices410 g,0.0117,0.500000
2,zehrs,White Bread,2.69,Old MillWhite Bread675 g,0.0040,0.666667
3,zehrs,Lean Ground Beef,10.00,Al SafaLean Ground Beef908 g,0.0110,0.666667


In [141]:
start_time = time.time()

store = 'zehrs'

regular_priced = pd.read_csv(f'clean_data/{store}/regular_prices.csv')
flyer = pd.read_csv(f'clean_data/{store}/flyer_deals.csv')

grocery_list = ['clementines', 'chicken breast', 'potatos', 'SHREDDED CHEESE']


final_selection = pd.DataFrame(columns = ['store', 'product_name', 'price'])

for item in grocery_list: 
    
    item_selection = pd.DataFrame(columns = ['store', 'product_name', 'price'])

    for index, row in flyer.iterrows():
        
        try: 
            product_name = row['product_name'].replace(',', '')
            price = row['price2']
            per_unit_price = row.per_unit_price2
    
            # find items  
            similarity = jaccard_similarity(item.lower().split(' '), product_name.lower().split(' '))
#             print(similarity, product_name)
            if similarity >= 0.3: # can tweak threshold but this is a good one for now  
                data = { 'store':store, 'product_name':product_name, 'price':price, 'per_unit_price': per_unit_price, 'simmilarity':similarity }
                item_selection = item_selection.append(data, ignore_index=True)
        except: continue

    try:
        # find lowest price from top similarities
        cheapest_item = item_selection.sort_values(by=['per_unit_price'])

        final_selection = final_selection.append(dict(cheapest_item.iloc[0]), ignore_index=True)
    except: continue
    
    
print(time.time() - start_time, 'seconds')

final_selection

0.1227271556854248 seconds


,store,product_name,price,per_unit_price,simmilarity
0,zehrs,LARGE CLEMENTINES,3.99,3.99,0.5
1,zehrs,ZERTO SHREDDED CHEESE 140 g,7.49,7.49,0.4


## pulling it all together 

In [162]:
start_time = time.time()

stores = ['zehrs', 'no_frills', 'valu_mart']

zehrs_results = pd.DataFrame()
no_frills_results = pd.DataFrame()
valu_mart_results = pd.DataFrame()

grocery_list = ['2% milk', 'Cheddar Cheese', 'white sliced bread', 'ground beef', 
                'clementines', 'chicken breast', 'potatos', 'shredded cheese']

for store in stores:
    regular_priced = pd.read_csv(f'clean_data/{store}/regular_prices.csv')
    flyer = pd.read_csv(f'clean_data/{store}/flyer_deals.csv')
    
    final_selection = pd.DataFrame(columns = ['list_item', 'store', 'product_name', 'price', 'per_unit_price', 'source'])
    
    for item in grocery_list: 
        item_selection = pd.DataFrame(columns = ['list_item', 'store', 'product_name', 'price', 'per_unit_price', 'source'])
        
        ##### search regular price data #####
        for index, row in regular_priced.iterrows():
            product_name = row['product']
            per_unit_price = row['per_unit_price2']
            price = row['price2']

            # find items  
            similarity = jaccard_similarity(item.lower().split(' '), product_name.lower().split(' '))

            if similarity >= 0.5: # can tweak threshold but this is a good one for now  
                data = { 'list_item':item, 'store':store, 'product_name':product_name, 'price':price, 'per_unit_price':per_unit_price, 'simmilarity':similarity, 'source': 'reg' }
                item_selection = item_selection.append(data, ignore_index=True)
    
    
        ##### search flyer data #####
        for index, row in flyer.iterrows():

            try: 
                product_name = row['product_name'].replace(',', '')
                price = row['price2']
                per_unit_price = row.per_unit_price2

                # find items  
                similarity = jaccard_similarity(item.lower().split(' '), product_name.lower().split(' '))
                
                if similarity >= 0.3: # can tweak threshold but this is a good one for now  
                    data = { 'list_item':item, 'store':store, 'product_name':product_name, 'price':price, 'per_unit_price': per_unit_price, 'simmilarity':similarity, 'source': 'flyer' }
                    item_selection = item_selection.append(data, ignore_index=True)
            except: continue


        try:
            # find lowest price from top similarities
            
            # ************** need to decide between per unit pricing and total price difference
            # ************** maybe some units need per unit some dont 
            cheapest_item = item_selection.sort_values(by=['per_unit_price'])
#             cheapest_item = item_selection.sort_values(by=['price'])

            final_selection = final_selection.append(dict(cheapest_item.iloc[0]), ignore_index=True)
        except: continue

            
    if store == 'zehrs':
        zehrs_results = final_selection
    elif store == 'no_frills':
        no_frills_results = final_selection
    elif store == 'valu_mart':
        valu_mart_results = final_selection  
    
print(time.time() - start_time, 'seconds')

3.6584599018096924 seconds


In [163]:
zehrs_results

,list_item,store,product_name,price,per_unit_price,source,simmilarity
0,2% milk,zehrs,2% Milk,4.99,0.00250,reg,1.000000
1,Cheddar Cheese,zehrs,Cheddar Cheese Thin Slices,4.79,0.01170,reg,0.500000
2,white sliced bread,zehrs,White Bread,2.69,0.00400,reg,0.666667
3,ground beef,zehrs,Lean Ground Beef,10.00,0.01100,reg,0.666667
4,clementines,zehrs,Organic Clementines,9.99,0.01100,reg,0.500000
5,chicken breast,zehrs,"Chicken Breast Fillets, Boneless",8.72,0.01982,reg,0.500000
6,shredded cheese,zehrs,Pizza Mozzarella Shredded Cheese,4.79,0.01500,reg,0.500000


In [164]:
no_frills_results

,list_item,store,product_name,price,per_unit_price,source,simmilarity
0,2% milk,no_frills,2% Milk,4.99,0.0025,reg,1.000000
1,Cheddar Cheese,no_frills,Cheddar Cheese Thin Slices,3.79,0.0092,reg,0.500000
2,white sliced bread,no_frills,White Bread,1.79,0.0027,reg,0.666667
3,ground beef,no_frills,Lean Ground Beef,10.00,0.0110,reg,0.666667
4,clementines,no_frills,Clementines,3.99,0.0017,reg,1.000000
5,chicken breast,no_frills,Chicken Breast Strips,10.99,0.0137,reg,0.666667
6,shredded cheese,no_frills,Shredded Marble Farmer's Cheese,13.99,0.0155,reg,0.500000


In [165]:
valu_mart_results

,list_item,store,product_name,price,per_unit_price,source,simmilarity
0,2% milk,valu_mart,2% Milk,4.99,0.0025,reg,1.000000
1,Cheddar Cheese,valu_mart,Cheddar Cheese Thin Slices,4.99,0.0122,reg,0.500000
2,white sliced bread,valu_mart,White Bread,2.79,0.0041,reg,0.666667
3,ground beef,valu_mart,Lean Ground Beef,4.98,0.0110,reg,0.666667
4,clementines,valu_mart,Clementines,6.99,0.0030,reg,1.000000
5,shredded cheese,valu_mart,Shredded Marble Farmer's Cheese,14.99,0.0167,reg,0.500000


In [166]:
zehrs_per_unit_subtotal = zehrs_results.per_unit_price.sum()
no_frills_per_unit_subtotal = no_frills_results.per_unit_price.sum()
valu_mart_per_unit_subtotal = valu_mart_results.per_unit_price.sum()

zehrs_subtotal = zehrs_results.price.sum()
no_frills_subtotal = no_frills_results.price.sum()
valu_mart_subtotal = valu_mart_results.price.sum()

lowest_price = min(zehrs_per_unit_subtotal, no_frills_per_unit_subtotal, valu_mart_per_unit_subtotal)

print(lowest_price)
print(f'Zehrs: {zehrs_per_unit_subtotal}, {zehrs_subtotal}')
print(f'No Frills: {no_frills_per_unit_subtotal}, {no_frills_subtotal}')
print(f'Valu Mart: {valu_mart_per_unit_subtotal}, {valu_mart_subtotal}')

0.0494999999999998
Zehrs: 0.0750199999999999, 45.97
No Frills: 0.0563, 49.540000000000006
Valu Mart: 0.0494999999999998, 39.730000000000004
